# Human Like Quesitons

In [51]:
import os
from openai import OpenAI
import pandas as pd

openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

#### Load Existing Questons

In [2]:
# from src.gdrive_api.utils import extract_questions, find_and_load_all_problems, extract_questions_by_topic

# parent_folder_path = './notebooks'

# all_problems = find_and_load_all_problems(parent_folder_path)
# questions = extract_questions(all_problems)
# existing_questions = list(questions)
# existing_questions

In [2]:
from src.gdrive_api.utils import extract_questions, find_and_load_all_problems, extract_questions_by_topic

parent_dir = "./notebooks"
all_problems = find_and_load_all_problems(parent_dir)
questions_grouped_by_topic = extract_questions_by_topic(all_problems)

# # Printing each topic and its questions
# for topic, questions in questions_grouped_by_topic.items():
#     print(f"Topic: {topic}")
#     for question in questions:
#         print(f" - {question}")

In [4]:

questions_by_topic = extract_questions_by_topic(all_problems)

#### Load Topic

In [3]:
import json

# Function to crawl the topic hierarchy and retrieve all topics
def crawl_keys(d, sep=' > ', prefix=''):
    paths = []
    for k, v in d.items():
        path = prefix + k
        if isinstance(v, dict) and len(v.keys()) > 0:
            paths.extend(crawl_keys(v, sep, path + sep))
        else:
            paths.append(path)
    return paths

# Open the `topic_hierarchy.json` file and retrieve all topics
with open('topic_hierarchy.json') as json_file:
    topic_hierarchy = json.load(json_file)

all_topics = crawl_keys(topic_hierarchy)
print(f"Total number of topics: {len(all_topics)}")

Total number of topics: 132


#### Max Questions to generate

In [5]:
MAX_QUESTIONS = 10  # The maximum number of questions
generated_questions_count = 0  # Counter for the number of questions generated so far

In [6]:
from src.gdrive_api.utils import generate_human_like_questions
from tqdm import tqdm

problems = []

questions_grouped_by_topics = extract_questions_by_topic(all_problems)

# If the existing questions are not grouped by full topic paths, convert them first
questions_grouped_by_topic = {}
for topic in all_topics:
    questions_grouped_by_topic[topic] = questions_grouped_by_topics.get(topic, set())

with tqdm(total=MAX_QUESTIONS) as pbar:
    for topic, existing_questions in questions_grouped_by_topics.items():
        # Stop if we've reached the max limit
        if generated_questions_count >= MAX_QUESTIONS:
            break

        # Generate questions
        questions = generate_human_like_questions(topic, 5, existing_questions)
        for question in questions["questions"]:
            # If we're at the max, break
            if generated_questions_count >= MAX_QUESTIONS:
                break
            problems.append({
                "metadata": {
                    "topic": topic,
                    "type": "query",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
            generated_questions_count += 1  # Increment for each question
            pbar.update(1)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


In [14]:

# from src.gdrive_api.utils import generate_human_like_questions
# from tqdm import tqdm

# problems = []

# with tqdm(total=MAX_QUESTIONS) as pbar:
#     for topic in all_topics:
#         # Stop if we've reached the max limit
#         if generated_questions_count >= MAX_QUESTIONS:
#             break

#         existing_questions = questions_grouped_by_topic.get(f'{topic}', set())
#         print('existing' + str(existing_questions))
#         print('topics' + str(topic))

#         # Generate questions
#         questions = generate_human_like_questions(topic, 5, existing_questions)
#         for question in questions["questions"]:
#             # If we're at the max, break
#             if generated_questions_count >= MAX_QUESTIONS:
#                 break
#             problems.append({
#                 "metadata": {
#                     "topic": topic,
#                     "type": "query",
#                     "difficulty": "Easy",
#                     "target_length": 1
#                 },
#                 "messages": [
#                     {"role": "user", "content": question},
#                 ]
#             })
#             generated_questions_count += 1  # Increment for each question
#             pbar.update(1)

In [8]:
from src.gdrive_api.utils import generate_human_like_code_modification_requests

generated_questions_count = 0

with tqdm(total=MAX_QUESTIONS) as pbar:
    for topic, existing_questions in questions_grouped_by_topics.items():
        # Stop if we've reached the max limit
        if generated_questions_count >= MAX_QUESTIONS:
            break
        
        questions = generate_human_like_code_modification_requests(topic, 3, existing_questions)
        for question in questions["questions"]:
            # If we're at the max, break
            if generated_questions_count >= MAX_QUESTIONS:
                break
            problems.append({
                "metadata": {
                    "topic": topic,
                    "type": "modification",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
            generated_questions_count += 1  # Increment for each question
            pbar.update(1)

100%|██████████| 100/100 [11:27<00:00,  6.88s/it]


In [9]:
problem_titles = []
problem_topic_counts = {}
file_path_to_problem = {}
for problem in problems:
    topic_type = f'{problem["metadata"]["topic"].split(" > ")[-1]}__{problem["metadata"]["type"]}'
    idx = problem_topic_counts.get(topic_type, 0)
    title = f'{topic_type}__{idx}'
    problem_titles.append(title)
    file_path_to_problem[f"{title}.ipynb"] = problem
    problem_topic_counts[topic_type] = idx + 1

#### distribution for the number of turns

In [12]:
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell
import re
import os
import random

# Define distribution for the number of turns
distribution = {1: 25, 2: 50, 6: 25}

# Ensure that the distribution sums up to 100%
assert sum(distribution.values()) == 100, "The distribution must sum to 100%"

def weighted_choice(choices):
    total = sum(weight for choice, weight in choices)
    r = random.uniform(0, total)
    upto = 0
    for choice, weight in choices:
        if upto + weight >= r:
            return choice
        upto += weight

def select_number_of_turns(distribution):
    # Normalize distribution values to probabilities
    total = sum(distribution.values())
    normalized_distribution = {k: v/total for k, v in distribution.items()}
    turns = weighted_choice(normalized_distribution.items())
    return turns

for p, t in zip(problems, problem_titles):
    # Determine the parity based on the last digit in the title
    title_number = int(re.search(r'\d+$', t).group())  # Extract the last number from the title

    turns = select_number_of_turns(distribution)
    
    parity = title_number % 2

    # Create a new notebook
    notebook = new_notebook()

    # Add metadata
    metadata = f"""# Metadata

**Python Topics** - {p["metadata"]["topic"]}

**Type** - {p["metadata"]["type"]}

**Target Number of Turns (User + Assistant)** - {turns}
"""
    metadata_cell = new_markdown_cell(metadata)
    notebook.cells.append(metadata_cell)

    # Add conversation header
    conversation_header = "# Conversation"
    conversation_header_cell = new_markdown_cell(conversation_header)
    notebook.cells.append(conversation_header_cell)

    # Append conversation messages
    title = "**User**" if parity else "**Assistant**"
    for message in p["messages"]:
        msg_content = f"""{title}

{message["content"]}
"""
        conversation_message_cell = new_markdown_cell(msg_content)
        notebook.cells.append(conversation_message_cell)

    # Save the notebook
    notebook_path = f'notebooks/test/{t}.ipynb'
     
    try:
        os.makedirs(os.path.dirname(notebook_path))
    except FileExistsError:
        pass
     
    with open(notebook_path, 'w') as f:
        nbformat.write(notebook, f)

#### biased_normal_distribution

In [14]:
import json
import nbformat
import os
import re
import numpy as np
from nbformat.v4 import new_notebook, new_markdown_cell
from scipy.stats import rankdata

def flatten_complexity_scores(data, key_prefix='', scores_dict=None):
    if scores_dict is None:
        scores_dict = {}
    for key, value in data.items():
        if isinstance(value, dict):
            if 'complexity' in value:
                scores_dict[f'{key_prefix}.{key}'.strip('.')] = value['complexity']
            else:
                new_prefix = f'{key_prefix}.{key}'.strip('.')
                flatten_complexity_scores(value, new_prefix, scores_dict)
        elif isinstance(value, int):
            scores_dict[key_prefix] = value
    return scores_dict

with open('./topic_dist.json') as f:
    complexity_data = json.load(f)

# Flatten the complexity data into a dictionary
complexity_scores = flatten_complexity_scores(complexity_data)

# Sort the complexity scores in descending order
sorted_by_complexity = dict(sorted(complexity_scores.items(), key=lambda item: item[1], reverse=True))

# Calculate the percentiles based on the sorted complexity scores
# Higher complexity will have higher rank and thus higher percentile
complexity_ranks = rankdata(list(sorted_by_complexity.values()), method='average')
topic_percentiles = {topic: rank / len(complexity_scores) for topic, rank in zip(sorted_by_complexity.keys(), complexity_ranks)}

def get_turn_range(turns):
    """
    Converts a target turn number into a soft target range.

    Parameters:
    - turns (int): Target turn number.

    Returns:
    - A tuple representing the soft target range (min_turns, max_turns).
    """
    min_turns = max(1, turns - 1)
    max_turns = min(15, turns + 3)
    return min_turns, max_turns

def biased_normal_distribution(input_number):
    """
    Samples a number from a normal distribution that is biased towards the lower end,
    with a specified range and standard deviation.

    Parameters:
    - input_number (int or float): A number, typically representing a percentile, between 0 and 100 that
      influences the mean of the normal distribution.

    Returns:
    - int: A sample from the biased normal distribution, truncated to the nearest integer.
    """
    # Constants for the distribution
    min_val, max_val = 1, 15
    std_dev = 2.5  # Fixed standard deviation

    # Base mean for input_number 50 (biased towards lower end)
    base_mean = 1

    # Adjust the mean based on the input number (linear mapping)
    mean = np.interp(input_number, [0, 100], [min_val, max_val])
    
    # Adjust mean for the bias towards lower numbers
    adjusted_mean = (mean + 3*base_mean) / 5

    # Sample from a normal distribution
    while True:
        sample = np.random.normal(adjusted_mean, std_dev)
        if min_val <= sample <= max_val:
            break

    return int(sample)

for p, t in zip(problems, problem_titles):

    metadata_topic_hierarchy = p["metadata"].get("topic") 
    normalized_topic = metadata_topic_hierarchy.replace(" > ", ".")
    print(normalized_topic)

    # Retrieve the percentile using the normalized topic string
    percentile = topic_percentiles.get(normalized_topic, 0)
    print(f"Topic: {normalized_topic}, Percentile: {percentile}")

    # Calculate turns using the biased_normal_distribution function
    turns = biased_normal_distribution(percentile * 100)  # Percentile scaled to 0-100 
    print('turns'+str(turns))

    min_turns, max_turns = get_turn_range(turns)
    print(f'Target Number of Turns: {min_turns}-{max_turns}')
    
    # Create a new notebook
    notebook = new_notebook()
    # Add metadata
    metadata = f"""# Metadata

**Python Topics** - {p["metadata"]["topic"]}

**Type** - {p["metadata"]["type"]}

**Target Number of Turns (User + Assistant)** - {min_turns}-{max_turns}
"""
    metadata_cell = new_markdown_cell(metadata)
    notebook.cells.append(metadata_cell)

    # Add conversation header
    conversation_header = "# Conversation"
    conversation_header_cell = new_markdown_cell(conversation_header)
    notebook.cells.append(conversation_header_cell)

    # Append conversation messages
    user_message = f"""**User**

{p["messages"][0]["content"]}
"""
    conversation_message_cell = new_markdown_cell(msg_content)
    notebook.cells.append(conversation_message_cell)

    # Save the notebook
    notebook_path = f'notebooks/test/{t}.ipynb'
     
    try:
        os.makedirs(os.path.dirname(notebook_path))
    except FileExistsError:
        pass
     
    with open(notebook_path, 'w') as f:
        nbformat.write(notebook, f)

algorithms.by_data_structure.arrays
Topic: algorithms.by_data_structure.arrays, Percentile: 0.03787878787878788
turns2
algorithms.by_data_structure.arrays
Topic: algorithms.by_data_structure.arrays, Percentile: 0.03787878787878788
turns2
algorithms.by_data_structure.arrays
Topic: algorithms.by_data_structure.arrays, Percentile: 0.03787878787878788
turns2
algorithms.by_data_structure.arrays
Topic: algorithms.by_data_structure.arrays, Percentile: 0.03787878787878788
turns4
algorithms.by_data_structure.arrays
Topic: algorithms.by_data_structure.arrays, Percentile: 0.03787878787878788
turns2
algorithms.by_data_structure.linked_lists
Topic: algorithms.by_data_structure.linked_lists, Percentile: 0.14015151515151514
turns2
algorithms.by_data_structure.linked_lists
Topic: algorithms.by_data_structure.linked_lists, Percentile: 0.14015151515151514
turns1
algorithms.by_data_structure.linked_lists
Topic: algorithms.by_data_structure.linked_lists, Percentile: 0.14015151515151514
turns2
algorithms.b

In [4]:
import csv

def parse_and_flatten_csv(input_csv_filepath, output_csv_filepath):
    with open(input_csv_filepath, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_csv_filepath, mode='w', newline='', encoding='utf-8') as outfile:
        
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        for row in reader:
            highlevel_topic = row[0].strip()
            subtopics = row[1].split('\n')
            for subtopic in subtopics:
                subtopic_cleaned = subtopic.strip()
                if subtopic_cleaned:
                    writer.writerow([highlevel_topic, subtopic_cleaned])

input_csv_filepath = './Copy of Copy of c.ai Turing Feedback - Areas to focus on.csv'  
output_csv_filepath = 'flattened.csv'

parse_and_flatten_csv(input_csv_filepath, output_csv_filepath)

In [50]:
import json
import csv


def process_sublevels(highlevel_topic, sublevels_dict, writer):
    for sublevel, nested in sublevels_dict.items():
        combined_sublevel = sublevel 
        if nested:
            for nested_topic in nested.keys():
                writer.writerow([highlevel_topic, f'{combined_sublevel}.{nested_topic}'])
        else:  
            writer.writerow([highlevel_topic, combined_sublevel])


with open('./topic_hierarchy.json', 'r') as json_file:
    data = json.load(json_file)


with open('topics_1.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(['highlevel_topic', 'sublevel_topic'])


    for highlevel_topic, sublevels in data.items():
        if sublevels: 
            process_sublevels(highlevel_topic, sublevels, writer)
        else:  
            writer.writerow([highlevel_topic, ''])

In [5]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account key file
SERVICE_ACCOUNT_FILE = 'creds/google__sa.json'

# The scopes required for the Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID of your spreadsheet
SPREADSHEET_ID = '1bsM3nz13BPPqCxmbLYFz5Ed9KOfV51hNoagBP8dP948'

# Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Specify the range and values to update
range_ = 'Areas status!A:B'  

def read_csv(filename):
    with open(filename, newline='') as csv_file:
        reader = csv.reader(csv_file)
        return list(reader)

# Call the Sheets API
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

# Load data from the CSV file
csv_values = read_csv('./flattened.csv')

# Use the Sheets API to write data
request = sheet.values().update(spreadsheetId=SPREADSHEET_ID, range=range_,
                               valueInputOption='RAW', body={'values': csv_values})
response = request.execute()

print(f"{len(csv_values) - 1} rows have been updated.")

30 rows have been updated.


In [15]:
with open('./topic_dist.json') as f:
    data = json.load(f)

# Flatten the data structure, creating a list of (category, complexity) tuples
def flatten_data(data, parent_key='', sep='_'):
    items = []
    for k, v in data.items():
        new_key = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            if 'complexity' in v:
                items.append((new_key, v['complexity']))
            else:
                items.extend(flatten_data(v, new_key, sep=sep))
    return items

# Flatten the provided JSON data
flattened_data = flatten_data(data)

# Now we sort the flattened data by complexity in descending order
sorted_data = sorted(flattened_data, key=lambda x: x[1], reverse=True)

for item in sorted_data:
    print(f"{item[0]}: Complexity {item[1]}")

algorithms_by_data_structure_advanced_data_structures: Complexity 8
algorithms_by_topic_dynamic_programming: Complexity 8
python_language_and_scripting_python_c_extensions: Complexity 8
deep_learning_backpropagation_understanding: Complexity 8
algorithms_by_data_structure_graphs: Complexity 7
algorithms_by_topic_famous_algorithms: Complexity 7
algorithms_by_topic_game_theory: Complexity 7
algorithms_by_topic_divide_and_conquer: Complexity 7
algorithms_by_topic_backtracking: Complexity 7
algorithms_by_topic_combinatorics: Complexity 7
python_language_and_scripting_metaclasses_and_class_factories: Complexity 7
python_language_and_scripting_memory_management_and_python_internals: Complexity 7
python_language_and_scripting_advanced_networking: Complexity 7
python_language_and_scripting_cython_and_PyPy: Complexity 7
python_language_and_scripting_parallel_programming: Complexity 7
unit_testing_methodology_test_ai_and_ml_models: Complexity 7
web_development_web_security: Complexity 7
ml_princ

In [19]:
import json

with open('notebooks/test/problems.json', 'w') as f:
    json.dump(problems, f, indent=4)

In [20]:
with open('notebooks/test/problems.json') as f:
    problems = json.load(f)

In [21]:
from tqdm import tqdm
from src.gdrive_api.folder_upload import upload_folder
from src.gdrive_api.auth import build_service

service = build_service('creds/google__sa.json')
# destination_folder_url = "https://drive.google.com/drive/u/0/folders/1Z1bdYMe2Qmo_vs-OaKDaYIiV3rIqLJH9"
# destination_folder_url = "https://drive.google.com/drive/u/2/folders/1sfPFHkXYpKyY41V0pfz3Qw3k4VLy5Hvb"
destination_folder_url = 'https://drive.google.com/drive/folders/1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'

uploaded_files = upload_folder(service, 'notebooks/test', destination_folder_url, force_replace = True, is_url=True)

file_path_to_url = {}
with tqdm(total=len(uploaded_files)) as pbar:
    for file_path, file_url in uploaded_files.items():
        if file_url is not None:
            drive_id = file_url.split("id=")[-1].split("&")[0].strip()
            colab_url = f"https://colab.research.google.com/drive/{drive_id}"
            file_path_to_url[file_path] = colab_url
        else:
            print(f"Skipped uploading {file_path}")
        pbar.update(1)

for file_path in file_path_to_url.keys():
    if file_path == "problems.json":
        continue
    problem = file_path_to_problem[file_path]
    problem["metadata"]["colab_url"] = file_path_to_url[file_path]
    problem["metadata"]["file_path"] = file_path
    problem["metadata"]["batch_idx"] = "test"

------------------------------------------------------------
Processing directory .: 1 of 0 in total.
Uploading file 1 of 41 in '.', 1 of 41 in total.
Uploading new file 'arrays__modification__0.ipynb'.
File 'arrays__modification__0.ipynb' has been uploaded.
Uploaded 'arrays__modification__0.ipynb' to folder ID '1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'.
arrays__modification__0.ipynb
Uploading file 2 of 41 in '.', 2 of 41 in total.
Uploading new file 'arrays__modification__1.ipynb'.
File 'arrays__modification__1.ipynb' has been uploaded.
Uploaded 'arrays__modification__1.ipynb' to folder ID '1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'.
arrays__modification__1.ipynb
Uploading file 3 of 41 in '.', 3 of 41 in total.
Uploading new file 'arrays__modification__2.ipynb'.
File 'arrays__modification__2.ipynb' has been uploaded.
Uploaded 'arrays__modification__2.ipynb' to folder ID '1T14cHENpx7j2aS4Yb_3WhnXhMql0zq2y'.
arrays__modification__2.ipynb
Uploading file 4 of 41 in '.', 4 of 41 in total.
Replacing exist

100%|██████████| 41/41 [00:00<?, ?it/s]


In [22]:
problems = list(file_path_to_problem.values())

with open('notebooks/test/problems.json', 'w') as f:
    json.dump(problems, f, indent=4)

In [23]:
# from google.oauth2 import service_account
# from googleapiclient.discovery import build

# # Path to your service account key file
# SERVICE_ACCOUNT_FILE = 'creds/google__sa.json'

# # The scopes required for the Sheets API
# SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# # The ID of your spreadsheet
# SPREADSHEET_ID = '1qBU7Kvuuij2fxbqPxebReKMxWgIBmOIE5Gi4ZuX0j_4'


# # Authenticate and build the service
# creds = service_account.Credentials.from_service_account_file(
#         SERVICE_ACCOUNT_FILE, scopes=SCOPES)
# service = build('sheets', 'v4', credentials=creds)

# # Specify the range and values to update
# range_ = 'Conversations_Batch_3!A2:E610'  # For example, this updates cells from A1 to D5 in Sheet1
# values = []

# for problem in problems:
#     values.append([
#         problem["metadata"]["colab_url"],
#         problem["metadata"]["topic"],
#         problem["metadata"]["target_length"],
#         problem["metadata"]["type"],
#         problem["metadata"]["batch_idx"],
#     ])


# body = {
#     'values': values
# }

# # Call the Sheets API to update the range
# request = service.spreadsheets().values().update(spreadsheetId=SPREADSHEET_ID, range=range_, valueInputOption='RAW', body=body)
# response = request.execute()

In [ ]:
# python related questions, add - student related types

#### create occupation with experience levels

In [55]:
import json

with open('./occupations.json') as json_file:
    topic_hierarchy = json.load(json_file)

def combine_occupations_with_experience(occupations_dict, experience_levels):
    occupation_pairs = []
    for occupation_category in occupations_dict:
        if occupation_category == "UserExperience":  # Skip experience levels key
            continue

        for occupation_type in ["MostPopular", "Trending"]:
            for occupation in occupations_dict[occupation_category].get(occupation_type, []):
                for level in experience_levels:
                    occupation_pairs.append(f"{occupation} > {level}")
    return occupation_pairs

# Extract the experience levels from the hierarchy
experience_levels = topic_hierarchy["UserExperience"]["Levels"]

# Combine occupations with experience levels
occupation_with_experience_pairs = combine_occupations_with_experience(topic_hierarchy, experience_levels)

print(f"Total number of occupation with experience levels: {len(occupation_with_experience_pairs)}")
print(occupation_with_experience_pairs)

Total number of occupation with experience levels: 200
['Python Developer > First Timer', 'Python Developer > Beginner', 'Python Developer > Intermediate', 'Python Developer > Advanced', 'Python Developer > Expert', 'Web Developer > First Timer', 'Web Developer > Beginner', 'Web Developer > Intermediate', 'Web Developer > Advanced', 'Web Developer > Expert', 'Data Scientist > First Timer', 'Data Scientist > Beginner', 'Data Scientist > Intermediate', 'Data Scientist > Advanced', 'Data Scientist > Expert', 'Mobile App Developer > First Timer', 'Mobile App Developer > Beginner', 'Mobile App Developer > Intermediate', 'Mobile App Developer > Advanced', 'Mobile App Developer > Expert', 'Python DevOps Engineer > First Timer', 'Python DevOps Engineer > Beginner', 'Python DevOps Engineer > Intermediate', 'Python DevOps Engineer > Advanced', 'Python DevOps Engineer > Expert', 'Front-End Developer > First Timer', 'Front-End Developer > Beginner', 'Front-End Developer > Intermediate', 'Front-End

In [75]:
occupation_skills = occupation_with_experience_pairs

In [72]:
print(type(occupation_with_experience_pairs))

<class 'list'>


In [53]:
df = pd.read_csv('./compatible_area.csv')

filtered_df = df[df['Status'] == 'To Cover']

print(len(filtered_df))

17


In [59]:
filtered_df['combined'] = filtered_df['Highlevel_topic'] + " - " + filtered_df['Sublevel_topic']

C:\Users\hp\AppData\Local\Temp\ipykernel_1832\2856107563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['combined'] = filtered_df['Highlevel_topic'] + " - " + filtered_df['Sublevel_topic']


In [73]:
areas_of_focus = filtered_df['combined'].to_list()

In [74]:
areas_of_focus

['Basic scripting - Write simple code in Javascript',
 'General coding help - Rewrite code using better formatting / best practises',
 'General coding help - Refactor code',
 'General coding help - Explain complex code snippets',
 'General coding help - Add comments to code',
 'General coding help - Do a code review',
 'Write unit test - Explain code with comments',
 'Interview preparation - Explain the problem step by step',
 'Interview preparation - Explain areas where people usually make mistakes',
 'Interview preparation - Explain alternative solutions and trade offs',
 'Interview preparation - Debug and fix code',
 'Interview preparation - System Design rounds',
 'Interview preparation - Proactively ask follow up questions',
 'Interview preparation - User asks AI help to take an Interview. AI takes an interview',
 'Debugging and Troubleshooting - Debug error trace',
 'Learning - Learn to program',
 'Learning - Learn to use some tool / infra']

In [ ]:
# 3. Suggestions should provide a clear direction for further development or learning.

#### matching Occupation and skill pairs with Compatible areas of focus

In [103]:
def compatible_area_check(occupation_skill, area_of_focus, n=1, existing_questions=None):
    SYSTEM_PROMPT = f"""IDENTITY:
You are an AI capable of identifying compatible areas of focus based on individuals' occupation and skills from the given list of compatible area only and if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible..

CONTEXT:
We are trying to assist users in aligning their professional expertise and their skills with the most compatible areas of focus. This will help in furthering their career development and staying relevant in their fields.
We already have a batch of existing occupation-skill-area of focus combinations and we want to avoid duplication when providing new suggestions.

CONSTRAINTS:
1. The suggestions must be from the input list of compatible area relevant to the provided occupation and skill level if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible..
2. It should be practical, meaning it can realistically be pursued given the user's background.


INSTRUCTION:
You will be given pairs of occupations and skills and and also list of selected compatible areas and will be asked to find a number of compatible areas of focus. You should also be provided with existing pairs and asked to ignore suggesting those if they already exist.
Act accordingly.

Find compatible areas of focus for the given occupation and skill pairs:
- Occupation and skill pairs: 
The suggestions should only be from the provided areas of focus if not compatible with occupation and skill from the list of area of focus please skip and only include the ones compatible.

RESPONSE FORMAT:
A JSON-valid object that includes {{"questions": ['Occupation and skill','area_of_focus1', 'Occupation and skill','area_of_focus2', ...] }}.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"Occupation and skill: {occupation_skill} \nCompatible areas of focus: {area_of_focus}"},
            ],
            temperature=0.0,
            max_tokens=4096,
            seed = 42,
            response_format={ 
                "type": "json_object" 
            },
        )
        questions = json.loads(response.choices[0].message.content)
        return questions
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
# total_iterations = len(occupation_skills) * len(areas_of_focus)

In [104]:
problems = []


with tqdm(total=len(occupation_skills)) as pbar:
    for occupation_skill in occupation_skills:
        for area_of_focus in areas_of_focus:

            matched_area = compatible_area_check(occupation_skill, area_of_focus, n=1)

            for question in matched_area["questions"]:
                print(question)
                problems.append({
                    "metadata": {
                        "occupation_skill": occupation_skill,
                        "compatible_area_of_focus": area_of_focus,
                        "type": "query",
                        "difficulty": "Easy",
                        "target_length": 1
                    },
                    "messages": [
                        {"role": "user", "content": question},
                    ]
                })
                pbar.update(1)

Python Developer > First Timer
General coding help - Rewrite code using better formatting / best practises


Python Developer > First Timer
General coding help


Python Developer > First Timer
General coding help


Python Developer > First Timer
General coding help


Python Developer > First Timer
General coding help


...

Data Engineer > Beginner
Debugging and Troubleshooting - Debug error trace


Data Engineer > Beginner
Learning - Learn to program


Data Engineer > Beginner
Learning - Learn to use some tool / infra


1074it [32:33,  1.82s/it]


KeyboardInterrupt: 

In [90]:
problems

[{'metadata': {'occupation_skill_pair': 'Python Developer > First Timer',
   'compatible_area_of_focus': 'Basic scripting - Write simple code in Javascript',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user', 'content': 'P'}]},
 {'metadata': {'occupation_skill_pair': 'Python Developer > First Timer',
   'compatible_area_of_focus': 'Basic scripting - Write simple code in Javascript',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user', 'content': 'y'}]},
 {'metadata': {'occupation_skill_pair': 'Python Developer > First Timer',
   'compatible_area_of_focus': 'Basic scripting - Write simple code in Javascript',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user', 'content': 't'}]},
 {'metadata': {'occupation_skill_pair': 'Python Developer > First Timer',
   'compatible_area_of_focus': 'Basic scripting - Write simple code in Javascript',
   'type

In [ ]:
with tqdm(total=len(all_topics)) as pbar:
    for topic in all_topics:
        questions = generate_human_like_code_modification_requests(topic, 3)
        for question in questions["questions"]:
            problems.append({
                "metadata": {
                    "occupation_skill_pair": occupation_skill,
                    "compatible_area_of_focus": area_of_focus,
                    "type": "query",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
        pbar.update(1)

In [ ]:
# filter

#### generating user themes based on their occupation and experience level

In [41]:
def generate_usecase_technical_topic(occupation, n=1, existing_questions=None):
    SYSTEM_PROMPT = f"""IDENTITY:
You are an AI capable of generating user themes based on their occupation and experience level.

CONTEXT:
We are trying to generate human-like queries that a user would send to an ai assistant through a chat interface. 
The user's query tone & structure should be diversified as much as possible making sure to include some realistic examples.
We already have batch of previously generated questions and we want to avoid duplication of questions when generating new batch.

CONSTRAINTS:
1. Occupation and skill Related
2. Easy (Solvable by a median developer in 15 minutes)
3. Questions should elicit a response that includes code.

INSTRUCTION:
You will be given an occupation and experience level and asked for number of Use Case coupled with Technical Topic and User Personality to generate.
You will also be given previously generated questions for respective topic and ask to ignore generating question if exist.
The aim is to maximize diversity.
Act accordingly.

Generate a theme object for a user with the following occupation and experience level:
- Occupation: 
- Experience level: 
The theme object should contain a use case, a technical topic, and the user's personality.

RESPONSE FORMAT:
A JSON-valid object that includes {{"questions":["use_case", "technical_topic", and "user_personality"]}}.

"""
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"Occupation: {occupation} \nNumber of questions: {n} \n existing questions {existing_questions}"},
            ],
            temperature=0.0,
            max_tokens=4096,
            seed = 42,
            response_format={ 
                "type": "json_object" 
            },
        )
        questions = json.loads(response.choices[0].message.content)
        return questions
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [44]:
from tqdm import tqdm

problems = []
with tqdm(total=MAX_QUESTIONS) as pbar:
    for Occupation in all_occupations_with_levels:
        questions = generate_usecase_technical_topic(Occupation, 1)
        print(questions)
        for question in questions["questions"]:
            problems.append({
                "metadata": {
                    "Occupation": Occupation,
                    "type": "query",
                    "difficulty": "Easy",
                    "target_length": 1
                },
                "messages": [
                    {"role": "user", "content": question},
                ]
            })
        pbar.update(1)


 10%|█         | 1/10 [00:05<00:53,  5.99s/it]

{'questions': [{'use_case': "Creating a simple 'Hello, World!' program", 'technical_topic': 'Basic syntax in Python', 'user_personality': 'Curious and eager to learn the fundamentals'}]}


 20%|██        | 2/10 [00:16<01:07,  8.41s/it]

{'questions': [{'use_case': 'Creating a simple REST API endpoint to retrieve user data', 'technical_topic': 'Building a RESTful service with Flask', 'user_personality': 'Curious and eager to learn, with a preference for clear, step-by-step explanations'}]}


 30%|███       | 3/10 [00:20<00:47,  6.81s/it]

...

{'questions': [{'use_case': 'Automating a simple deployment process', 'technical_topic': 'Writing a basic Ansible playbook', 'user_personality': 'Eager to learn and apply new tools'}]}


23it [02:01,  5.05s/it]

{'questions': [{'use_case': 'Automating deployment processes', 'technical_topic': 'Writing a basic Ansible playbook', 'user_personality': 'Curious and eager to optimize workflows'}]}


23it [02:04,  5.43s/it]


KeyboardInterrupt: 

In [47]:
problems

[{'metadata': {'Occupation': 'SoftwareCentricOccupations > MostPopular > Software Developer > First Timer',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content': {'use_case': "Creating a simple 'Hello, World!' program",
     'technical_topic': 'Basic syntax in Python',
     'user_personality': 'Curious and eager to learn the fundamentals'}}]},
 {'metadata': {'Occupation': 'SoftwareCentricOccupations > MostPopular > Software Developer > Beginner',
   'type': 'query',
   'difficulty': 'Easy',
   'target_length': 1},
  'messages': [{'role': 'user',
    'content': {'use_case': 'Creating a simple REST API endpoint to retrieve user data',
     'technical_topic': 'Building a RESTful service with Flask',
     'user_personality': 'Curious and eager to learn, with a preference for clear, step-by-step explanations'}}]},
 {'metadata': {'Occupation': 'SoftwareCentricOccupations > MostPopular > Software Developer > Intermediate',
   'type